In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
clear = lambda: os.system('clear')


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anjali/AnjaliOldLipi-Regular.woff
/kaggle/input/anjali/AnjaliOldLipi-Regular.woff2
/kaggle/input/anjali/AnjaliOldLipi-Regular.ttf
/kaggle/input/aksharantar/aksharantar_sampled/brx/brx_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/brx/brx_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/brx/brx_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/tam/tam_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/tam/tam_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/tam/tam_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/mni/mni_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/mni/mni_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/mni/mni_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/urd/urd_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/urd/urd_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/urd/urd_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/kok/kok_valid.csv
/kaggle/input/aksharantar/ak

### ALL NECESSARY IMPORTS

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

from tqdm import tqdm
import heapq

import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

import wandb
# Instantiates the device to be used as GPU/CPU based on availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#specify max length of sequence
mal_embedd_size = 29
eng_embedd_size = 32
device.type

'cuda'

### PREPROCESSING STEP

In [5]:
# Load Data to capture all characters
arr = np.loadtxt("/kaggle/input/aksharantar/aksharantar_sampled/mal/mal_train.csv",
                 delimiter=",", dtype=str)
num_sample = arr.shape[0]
x_train,y_train = arr[:,0],arr[:,1]
english_index = 3
mal_index = 3
english_dict = {}
malayalam_dict = {}
english_index_dict = {}
malayalam_index_dict = {}

# Create dictionary for malayalam and english

for i in range(num_sample):
    for j in range(len(x_train[i])):
        
        if(english_dict.get(x_train[i][j]) == None):
            english_dict[x_train[i][j]]=english_index
            english_index_dict[english_index] = x_train[i][j]
            english_index+=1
        
    for j in range(len(y_train[i])):
            
        if(malayalam_dict.get(y_train[i][j]) == None):
            malayalam_dict[y_train[i][j]]=mal_index
            malayalam_index_dict[mal_index] = y_train[i][j]
            mal_index+=1

In [6]:
# Adding start, stop and padding symbols
malayalam_index_dict[1] = '<S>'
english_index_dict[1] = '<S>'

malayalam_index_dict[2] = '<E>'
english_index_dict[2] = '<E>'

malayalam_index_dict[0] = '<P>'
english_index_dict[0] = '<P>'

In [7]:
# Function to load data from the data set and create and array corresponding to it

def loadData(PATH = "/kaggle/input/aksharantar/aksharantar_sampled/mal/mal_test.csv"):  
    
    arr = np.loadtxt(PATH,
                 delimiter=",", dtype=str)
    num_sample = arr.shape[0]
    x,y = arr[:,0],arr[:,1]
    X = np.zeros((num_sample,eng_embedd_size))
    Y = np.zeros((num_sample,mal_embedd_size))

    for i in range(num_sample):

        X[i][0] = 1
        Y[i][0] = 1
        for j in range(len(x[i])):
            if(english_dict.get(x[i][j]) != None):
                X[i][j+1] = english_dict[x[i][j]]
            else:
                X[i][j+1] = 0

        X[i][len(x[i])+1]=2

        for j in range(len(y[i])):
            if(malayalam_dict.get(y[i][j]) != None):
                Y[i][j+1] = malayalam_dict[y[i][j]]
            else:
                Y[i][j+1] = 0

        Y[i][len(y[i])+1] = 2
        
    return X, Y

In [8]:
# Load all data with specified path
X_train,y_train = loadData(PATH = "/kaggle/input/aksharantar/aksharantar_sampled/mal/mal_train.csv")
X_val,y_val = loadData(PATH = "/kaggle/input/aksharantar/aksharantar_sampled/mal/mal_valid.csv")
X_test,y_test = loadData(PATH = "/kaggle/input/aksharantar/aksharantar_sampled/mal/mal_test.csv")

In [9]:
# Class to create dataset, so can be passed to pytorch dataloader
class MakeDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.int64)
        self.y = torch.tensor(y,dtype=torch.int64)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]

  
    def __len__(self):
        return self.len

In [19]:
# Create dataset to pass to dataloader

train_dataset = MakeDataset(X_train[:1000],y_train[:1000])
val_dataset = MakeDataset(X_val[:1000], y_val[:1000])
test_dataset = MakeDataset(X_test, y_test)

In [20]:
# Create dataloader so getting data in epochs is easy
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=256)
val_loader = DataLoader(val_dataset,shuffle=True,batch_size=256)
test_loader = DataLoader(test_dataset,shuffle=True,batch_size=256)

## ENCODER MODULE

In [21]:
class Encoder(nn.Module):
    '''
     Encoder network in which hidden size, input dimension, embedding dimension, etc 
     can be specified for training.
    '''
    def __init__(self,
                    input_dimension = 72,
                    embed_dimension = 64,
                    hidden_dimension = 256,
                    rnn_type = 'gru',
                    layers = 2,
                    bidirectional = True,
                    dropout = 0,
                    device = device
                ):
        
        super(Encoder, self).__init__()

        self.detail_parameters = {
            'input_dimension' : input_dimension,
            'embed_dimension' : embed_dimension,
            'hidden_dimension' : hidden_dimension,
            'rnn_type' : rnn_type,
            'dropout' : dropout,
            'layers' : layers,
            'direction_value' : 2 if bidirectional else 1,
            'device' : device.type,
        }
        
        self.input_dimension = input_dimension
        self.embed_dimension = embed_dimension
        self.hidden_dimension = hidden_dimension
        self.rnn_type = rnn_type
        self.dropout = nn.Dropout(dropout)
        self.layers = layers
        self.direction_value = 2 if bidirectional else 1
        self.device = device

        self.embedding = nn.Embedding(self.input_dimension, self.embed_dimension)

        if self.rnn_type == 'rnn':
            self.enc_rnn = nn.RNN(
                          input_size= self.embed_dimension,
                          hidden_size= self.hidden_dimension,
                          num_layers= self.layers,
                          dropout = dropout,
                          bidirectional= bidirectional)
            
        elif self.rnn_type == "gru":
            self.enc_rnn = nn.GRU(
                          input_size= self.embed_dimension,
                          hidden_size= self.hidden_dimension,
                          num_layers= self.layers,
                          dropout = dropout,
                          bidirectional= bidirectional)
        elif self.rnn_type == "lstm":
            self.enc_rnn = nn.LSTM(
                          input_size= self.embed_dimension,
                          hidden_size= self.hidden_dimension,
                          num_layers= self.layers,
                          dropout = dropout,
                          bidirectional= bidirectional)
            
            
    def forward(self, input, hidden, cell=None):
        
        # First convert sequence to embedding
        
        embedded = self.dropout(self.embedding(input))
        
        #Then choose type of rnn to run using pytorch 
        if self.rnn_type == 'lstm':
            output,(hidden,cell) = self.enc_rnn(embedded, (hidden,cell))
        else:
            output, hidden = self.enc_rnn(embedded, hidden)

        return output, hidden, cell

    def getParams(self):
        return self.detail_parameters
    
    def init_hidden(self, batch):
        # Initialize the hidden state to zeros
        return torch.zeros(self.direction_value*self.layers,batch,self.hidden_dimension,device=device)
        
    

## DECODER MODULE

In [22]:
class Decoder(nn.Module):
    '''
    Decoder to decode to malayalam word. It also contain different parameters
    which is specified in the contructor. 
    '''
    def __init__(self,
                input_dimension = 26,
                embed_dimension = 64,
                hidden_dimension = 256,
                rnn_type = 'gru',
                layers = 2,
                use_attention = False,
                attention_dimension = None,
                dropout = 0,
                bidirectional = True,
                device = device
                 ):
        
        super(Decoder, self).__init__()
        
        self.detail_parameters = {
            'input_dimension' : input_dimension,
            'embed_dimension' : embed_dimension,
            'hidden_dimension' : hidden_dimension,
            'rnn_type' : rnn_type,
            'layers' : layers,
            'device' : device.type,
            'dropout' : dropout
        }

        self.input_dimension = input_dimension
        self.hidden_dimension = hidden_dimension
        self.embed_dimension = embed_dimension
        self.rnn_type = rnn_type
        self.layers = layers
        self.use_attention = use_attention
        self.device = device
        self.dropout = nn.Dropout(dropout)
        self.directions = 2 if bidirectional else 1
        self.enc_outstate_dim = attention_dimension
        if self.use_attention:
            self.attention_dimension = attention_dimension if attention_dimension else hidden_dimension
        else:
            self.attention_dimension = 0


        self.embedding = nn.Embedding(self.input_dimension, self.embed_dimension)
    
        self.softmax = F.softmax
        self.fc = nn.Sequential(
            nn.Linear(self.hidden_dimension*self.directions, self.embed_dimension), nn.LeakyReLU(),
            nn.Linear(self.embed_dimension, self.input_dimension),
            )
        
        if self.use_attention:
            self.W1 = nn.Linear( self.attention_dimension, self.hidden_dimension)
            self.W2 = nn.Linear( self.hidden_dimension, self.hidden_dimension)
            self.V = nn.Linear( self.hidden_dimension, 1)

        if self.rnn_type == 'rnn':
            self.dec_rnn = nn.RNN(input_size= self.embed_dimension + self.attention_dimension,
                            dropout = dropout,     
                            hidden_size= self.hidden_dimension, # previous Hidden
                            num_layers= self.layers,
                            bidirectional= bidirectional
                                 )
        elif self.rnn_type == 'gru':
            self.dec_rnn = nn.GRU(input_size= self.embed_dimension + self.attention_dimension, # to concat attention_output
                            hidden_size= self.hidden_dimension, # previous Hidden
                            num_layers= self.layers,
                            dropout = dropout,
                            bidirectional= bidirectional
                                 )
        elif self.rnn_type == "lstm":
            self.dec_rnn = nn.LSTM(input_size= self.embed_dimension + self.attention_dimension, # to concat attention_output
                            hidden_size= self.hidden_dimension, # previous Hidden
                            num_layers= self.layers,
                            dropout = dropout,
                            bidirectional= bidirectional
                                  )


    def getParams(self):
        return self.detail_parameters
    
    def attention(self, hidden, enc_output):
        
        '''
        It uses attention mechanism to include encoders weights to decoder.
        '''
        encoder_transform  = self.W1(enc_output)
        hidden_transform =  self.W2(hidden)

        score = torch.tanh(encoder_transform + hidden_transform)

#         This will be our probability distribution for the attention weights (lambda)
        attention_weights = torch.softmax(self.V(score), dim=0)
        
#         conext vector to be concatenated to input
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector,dim=0)
        context_vector = torch.sum(context_vector,dim=0).unsqueeze(0)
        return context_vector,attention_weights
    
    def forward(self, input, hidden, cell=None,encoder_outputs=None):
#         Incorporate dropout in embedding.
        output = self.dropout(self.embedding(input))
    
        attention_weights = None
#         If we are using attention, then we need to concatenate the context vector, which we obtain from attention
        
        if self.use_attention:
            context,attention_weights = self.attention(hidden, encoder_outputs)
            output = torch.cat((output,context),2)
        
        if self.rnn_type == 'lstm':
            output,(hidden,cell) = self.dec_rnn(output,(hidden,cell))
        else:
            output, hidden = self.dec_rnn(output, hidden)
        output = self.fc(output)
        
        return output, hidden, cell, attention_weights

    

In [23]:
class BeamNode(object):
    def __init__(self, probability = 1, path_probability = 0, index = 1, hidden = None, cell = None, parent = None):
        self.probability = probability
        self.path_probability = path_probability
        self.index = index
        self.parent = parent
        self.hidden = hidden
        self.cell = cell
        self.length = 0
    
    def __lt__(self, other):
        return self.path_probability > other.path_probability

In [24]:
class BeamSearch():
    def __init__(self, beam_width = 3):
        self.open_list = []
        heapq.heapify(self.open_list)
        self.paths = []
        self.beam_width = beam_width 
    def beamSearch(self, model, outputs,dec_hiddens,cells, predicted):
        batch_size = outputs.shape[1]
        
        for i in range(batch_size):
            with torch.no_grad():
                model.eval()
                output = outputs[:,i:i+1].contiguous()
                index = output.contiguous()
                dec_hidden = dec_hiddens[:,i:i+1,:].contiguous()
                cell = cells[:,i:i+1,:].contiguous() if cells is not None else None
                node = BeamNode(1,1,index,dec_hidden, cell, None)
                heapq.heappush(self.open_list,node)

                while(len(self.open_list) > 0):
                    curr_node = heapq.heappop(self.open_list)
                    
                    if curr_node.length == model.output_seq_length-1:
                        self.paths.append(curr_node)
                        continue

                    output,dec_hidden,cell,attention_weights=model.decoder.forward(curr_node.index,curr_node.hidden,curr_node.cell,None)
                    output = model.softmax(output,dim=2)

                    topk, topk_index = torch.topk(output,self.beam_width, dim = 2)

                    for j in range(self.beam_width):
                        output = topk[:,:,j]
                        index = topk_index[:,:,j]
                        node = BeamNode(output.item(),curr_node.path_probability * output.item(),index,dec_hidden, cell, curr_node)
                        node.length = curr_node.length+1
                        heapq.heappush(self.open_list,node)



                    self.open_list = heapq.nsmallest(self.beam_width, self.open_list)

                path = min(self.paths)
                self.paths = []
            
                prev = None
                current = path
                while(current is not None):
                    next = current.parent
                    current.parent = prev
                    prev = current
                    current = next
                path = prev
                
            model.train()
            for t in range(1,model.output_seq_length):
                output,dec_hidden,cell,attention_weights=model.decoder.forward(path.index,path.hidden,path.cell,None)
                
                predicted[t,i:i+1] = output
                
                path = path.parent
                
                
#         return predicted

In [25]:
# Scoring function, which is used to calculate how many words in the batch is getting 100% word match
# This function is used in calculating accuracy
def scoring(y_dash , y):
    num_sample,seq_len = y.shape
    score = torch.sum(torch.sum(y_dash == y,axis = 1) == seq_len)
    return score

## SEQUENCE TO SEQUENCE MODULE

In [26]:
class Seq2Seq(nn.Module):
    
    '''
    This class incorporate the whole transliteration model. It calls encoder and pass output of encoder
    to decoder with or wihout attention. Parameters are specified in constructor.
    '''
    
    def __init__(self, 
                 input_seq_length = 32,
                 output_seq_length = 29,
                 encoder_input_dimension = 29, 
                 decoder_input_dimension = 72,
                 encoder_hidden_dimension = 256, 
                 decoder_hidden_dimension =256,
                 encoder_embed_dimension = 256, 
                 decoder_embed_dimension = 256, 
                 bidirectional = True,
                 encoder_num_layers = 3,
                 decoder_num_layers = 2,
                 rnn_type = 'lstm', 
                 dropout = 0.2, 
                 beam_width = 2,
                 device = device,
                 attention = False
                ):
        
        
        super(Seq2Seq, self).__init__()
        self.detail_parameters = {
         'input_seq_length': input_seq_length,
         'output_seq_length' : output_seq_length,
         'encoder_input_dimension' : encoder_input_dimension, 
         'decoder_input_dimension' : decoder_input_dimension,
         'encoder_hidden_dimension' : encoder_hidden_dimension,
         'encoder_embed_dimension' : encoder_embed_dimension, 
         'decoder_hidden_dimension':decoder_hidden_dimension,
         'decoder_embed_dimension' : decoder_embed_dimension, 
         'bidirectional' : bidirectional,
         'encoder_num_layers' : encoder_num_layers,
         'decoder_num_layers' : decoder_num_layers,
         'rnn_type' :rnn_type, 
         'dropout' : dropout, 
         'device' : device.type
        }
        self.input_seq_length = input_seq_length
        self.output_seq_length = output_seq_length
        self.encoder_input_dimension = encoder_input_dimension
        self.decoder_input_dimension = decoder_input_dimension
        self.encoder_hidden_dimension = encoder_hidden_dimension
        self.decoder_hidden_dimension = decoder_hidden_dimension
        self.encoder_embed_dimension = encoder_embed_dimension
        self.decoder_embed_dimension = decoder_embed_dimension 
        self.direction = bidirectional
        self.direction_value = 2 if bidirectional else 1
        self.encoder_num_layers = encoder_num_layers
        self.decoder_num_layers = decoder_num_layers
        self.rnn_type = rnn_type 
        self.dropout = dropout
        self.device = device
        self.softmax = F.softmax
        self.beam_width = beam_width
        self.use_attention = attention
        self.enc_dec_linear1 = nn.Linear(encoder_hidden_dimension,decoder_hidden_dimension)
        self.enc_dec_linear2 = nn.Linear(encoder_num_layers*self.direction_value,decoder_num_layers*self.direction_value)
        self.encoder = Encoder(input_dimension = self.encoder_input_dimension,
                               embed_dimension = self.encoder_embed_dimension, 
                               hidden_dimension =  self.encoder_hidden_dimension,
                               rnn_type = self.rnn_type,
                               layers = self.encoder_num_layers,
                               bidirectional = self.direction,
                               dropout = self.dropout, 
                               device = self.device
                              )
        self.decoder = Decoder(
                               input_dimension = self.decoder_input_dimension,
                               embed_dimension = self.decoder_embed_dimension,
                               hidden_dimension = self.decoder_hidden_dimension,
                               attention_dimension = self.encoder_hidden_dimension,
                               rnn_type = self.rnn_type,
                               layers = self.decoder_num_layers,
                               dropout = self.dropout, 
                               device = self.device,
                                use_attention = self.use_attention
                               )
        
    def getParams(self):
        return self.detail_parameters
    
    def forward(self, input, target ,teacher_force, acc_calculate = False):
        
        batch_size = input.shape[0]
        
        enc_hidden = self.encoder.init_hidden(batch_size)
        if self.rnn_type == 'lstm':
            cell = self.encoder.init_hidden(batch_size)
        else:
            cell = None
            
        encoder_outputs = None
        if self.use_attention:
            encoder_outputs = torch.zeros(self.input_seq_length,self.direction_value*self.decoder_num_layers,batch_size,self.encoder_hidden_dimension,device=device)
        
        
        for t in range(self.input_seq_length):
            enc_output,enc_hidden, cell = self.encoder.forward(input[:,t].unsqueeze(0), enc_hidden, cell)
            
            if self.use_attention:
                enc_hidden_new = enc_hidden
                enc_hidden_new = enc_hidden_new.permute(2,1,0).contiguous()
                enc_hidden_new = self.enc_dec_linear2(enc_hidden_new)
        #         dec_hidden = F.relu(dec_hidden)
                enc_hidden_new = enc_hidden_new.permute(2,1,0).contiguous()
                encoder_outputs[t] = enc_hidden_new
        
        enc_last_state = enc_hidden
        predicted = torch.zeros(self.output_seq_length, batch_size, self.decoder_input_dimension,device = self.device)
        attn_weights = torch.zeros(self.output_seq_length, self.input_seq_length, self.direction_value*self.decoder_num_layers ,batch_size, device = self.device)
        dec_hidden = enc_last_state
        dec_hidden = self.enc_dec_linear1(dec_hidden)
#         dec_hidden = F.relu(dec_hidden)


        dec_hidden = dec_hidden.permute(2,1,0).contiguous()
        dec_hidden = self.enc_dec_linear2(dec_hidden)
#         dec_hidden = F.relu(dec_hidden)
        dec_hidden = dec_hidden.permute(2,1,0).contiguous()
        
        if  self.rnn_type == 'lstm':
    #         cell = cell
    #         cell = self.enc_dec_linear1(cell)
    # #         cell = F.relu(cell)
            cell = cell.permute(2,1,0).contiguous()
            cell = self.enc_dec_linear2(cell)
    #         cell = F.relu(cell)
            cell = cell.permute(2,1,0).contiguous()

        
        output = torch.ones(1,batch_size,dtype=torch.long, device=self.device)
        predicted[0,:,1]=torch.ones(batch_size)
        attention_weights = None
        
        
            
        for t in range(1,self.output_seq_length):
            if teacher_force:
                output,dec_hidden,cell,attention_weights=self.decoder.forward(target[:,t-1].unsqueeze(0),dec_hidden,cell,encoder_outputs)
                predicted[t] = output.squeeze(0)

            else:
                if self.beam_width > 1 and acc_calculate:
                    beam = BeamSearch()
                    beam.beamSearch(self, output,dec_hidden,cell, predicted)
                    break
                    
                output,dec_hidden,cell,attention_weights=self.decoder.forward(output,dec_hidden,cell,encoder_outputs)
                predicted[t] = output.squeeze(0)
                if self.use_attention:
                    attn_weights[t] = attention_weights.squeeze(3)
                output = self.softmax(output,dim=2)
                output = torch.argmax(output,dim=2)

        
        return predicted,attn_weights

## TRAINING FOR 1 EXAMPLE CONFIGURATION

In [27]:
model = Seq2Seq()
model.to(device)
epochs = 10

def train(data_loader, val_loader ,epochs):
        
        optimizer = optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss()
        # set model to train mode
        model.train()
        attention_weights = None
        for epoch in tqdm(range(epochs)):
            total_loss=0
            train_loss = 0
            train_score = 0
            val_score = 0
            val_loss = 0
            for i, (source, target) in enumerate(data_loader):

                source = source.to(device)
                target = target.to(device)
                
                if i%10 == 0:
                    print(f'batch = > {i}')
                
                optimizer.zero_grad()
                
                output,attention_weights = model.forward(source, target, epoch < epochs/2, True)                

                output = output.permute(1, 0, 2)
                expected = F.one_hot(target,num_classes = 72).float()
                    
                output = output.reshape(-1, 72)

                expected = expected.reshape(-1,72)

                loss = criterion(output, expected)
                
                loss.backward()  # compute gradients
                nn.utils.clip_grad_norm_(model.parameters(),1)
                optimizer.step()  # update parameters
                
                clear()
#                 total_loss += loss.item()
 
#                 break
#             continue
            with torch.no_grad():
                model.eval()
                for val_input, val_target in val_loader:
                    val_input = val_input.to(device)
                    val_target = val_target.to(device)
                    val_output,_ = model.forward(val_input, None,False)
                    
                    acc_output = F.softmax(val_output,dim=2)
                    acc_output = torch.argmax(acc_output,dim=2)
                    acc_output = acc_output.T
                    val_score += scoring(acc_output,val_target)

                    
                    val_output = val_output.permute(1, 0, 2)
                    expected = F.one_hot(val_target,num_classes = 72).float()

                    val_output = val_output.reshape(-1, 72)

                    expected = expected.reshape(-1,72)

                    
                    loss = criterion(val_output, expected)
                    val_loss += loss.item()
                
            with torch.no_grad():
                model.eval()
                for train_input, train_target in data_loader:
                    train_input = train_input.to(device)
                    train_target = train_target.to(device)
                    train_output,_ = model.forward(train_input, None,False)
                    
                    acc_output = F.softmax(train_output,dim=2)
                    acc_output = torch.argmax(acc_output,dim=2)
                    acc_output = acc_output.T
                    train_score += scoring(acc_output,train_target)

                    
                    train_output = train_output.permute(1, 0, 2)
                    expected = F.one_hot(train_target,num_classes = 72).float()

                    train_output = train_output.reshape(-1, 72)

                    expected = expected.reshape(-1,72)

                    
                    loss = criterion(train_output, expected)
                    train_loss += loss.item()
                model.train()
            
                
                
            print(f'epoch {epoch}')
            print(f'train loss => {train_loss/len(data_loader)} \ntrain_acc => {train_score/len(data_loader.dataset)}')
            print(f'valid loss => {val_loss/len(val_loader)} \nvalid_acc => {val_score/len(val_loader.dataset)}')

train(train_loader, val_loader, epochs)

  0%|          | 0/10 [00:00<?, ?it/s]

batch = > 0


 10%|█         | 1/10 [00:00<00:08,  1.09it/s]

epoch 0
train loss => 2.5927923917770386 
train_acc => 0.0
valid loss => 2.037275642156601 
valid_acc => 0.0
batch = > 0


 20%|██        | 2/10 [00:01<00:07,  1.10it/s]

...

epoch 7
train loss => 2.1270986795425415 
train_acc => 0.0
valid loss => 2.120113730430603 
valid_acc => 0.0
batch = > 0


 90%|█████████ | 9/10 [11:07<01:53, 113.44s/it]

epoch 8
train loss => 2.3038485050201416 
train_acc => 0.0
valid loss => 1.7376804649829865 
valid_acc => 0.0
batch = > 0


100%|██████████| 10/10 [14:02<00:00, 84.25s/it] 

epoch 9
train loss => 2.535341441631317 
train_acc => 0.0
valid loss => 1.801782876253128 
valid_acc => 0.0


## TESTING AND ATTENTION PLOTS

In [47]:
# Get test examples, so we can plot the attention part - heatmap
# test_input, test_labels = next(iter(test_loader))
# model.eval()
# test_output,_weights = model.forward(test_input.to(device), None,False)

In [50]:
def getTicks(test_input, acc_output, data_index):
    x_t = []
    y_t = []
    for i in range(len(test_input[data_index])):
        if(test_input[data_index][i].item() != 0 and test_input[data_index][i].item() != 1 and test_input[data_index][i].item() != 2):
            x_t.append(english_index_dict[test_input[data_index][i].item()])

    for i in range(len(acc_output[data_index])):
        if(acc_output[data_index][i].item() != 0 and acc_output[data_index][i].item() != 1 and acc_output[data_index][i].item() != 2):
            y_t.append(malayalam_index_dict[acc_output[data_index][i].item()])
            
    return x_t, y_t
def plotHeatMap(test_input,acc_output,w,num_plots = 12):
    fig, ax = plt.subplots(4, 3,figsize=(20, 20))
    _ = plt.setp(ax)
    for data_index in range(num_plots):
        
        x_t, y_t = getTicks(test_input, acc_output, data_index)
        a = w[:,:,data_index]
        a = a.detach().cpu().numpy()
        a = a[1:len(y_t)+1,1:len(x_t)+1] 
        
        
        plt.sca(ax[data_index//3,data_index%3])
        
        
        sns.heatmap(a)
        plt.xticks(np.arange(0.5, len(x_t)+0.5), x_t)
        mal_font = FontProperties(fname = '/kaggle/input/anjali/AnjaliOldLipi-Regular.ttf')
        plt.yticks(np.arange(0.5, len(y_t)+0.5), y_t,fontproperties= mal_font)
        
        
        
#         plt.imshow(a,cmap='winter', interpolation='nearest')
#         plt.colorbar()
#         plt.xticks(np.arange(0, len(x_t)), x_t)
#         mal_font = FontProperties(fname = '/kaggle/input/anjali/AnjaliOldLipi-Regular.ttf')
#         plt.yticks(np.arange(0, len(y_t)), y_t,fontproperties= mal_font)
        
        plt.xlabel('English')
        plt.ylabel('Malayalam')
        plt.title(f'test image {data_index + 1}')
        
        
#         plt.show()

    plt.show()

# if model.use_attention:
#     acc_output = F.softmax(test_output,dim=2)
#     acc_output = torch.argmax(acc_output,dim=2)
#     acc_output.shape
#     acc_output = acc_output.T

#     w =  torch.mean(_weights,axis=2)
    
#     plotHeatMap(test_input,acc_output,w)
# else:
#     print("No Attention => No heatmap")



In [51]:
# def printOutput(y):
#     s=''
#     for i in range(len(y)):
#         if y[i] == 0 or y[i] == 1 or y[i] == 2:
#             continue
#         s+=malayalam_index_dict[y[i]]
#     return s

# print(printOutput(train_loader.dataset.dataset.y[1].detach().cpu().numpy()))
# printOutput(output[1].detach().cpu().numpy())
# malayalam_index_dict[train_loader.dataset.y[0][1]]


## WANDB SWEEPS RUN FUNCTION

In [52]:
def runModel(model, data_loader, val_loader ,epochs):
        
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()

    # set model to train mode
    model.train()
    train_loss_list = []
    val_loss_list = []
    train_accuracy_list = []
    val_accuracy_list = []

    for epoch in tqdm(range(epochs)):
        total_loss=0
        train_loss = 0
        train_score = 0
        val_score = 0
        val_loss = 0
        for i, (source, target) in enumerate(data_loader):
#                 print(f'batch {i}')

            source = source.to(device)
            target = target.to(device)

#                 print(target[:,1].shape)

            optimizer.zero_grad()

            output,_ = model.forward(source, target, epoch < epochs/2)                

#                 acc_output = F.softmax(output,dim=2)
#                 acc_output = torch.argmax(acc_output,dim=2)
#                 acc_output = acc_output.T
#                 train_score += scoring(acc_output,target)

            output = output.permute(1, 0, 2)
            expected = F.one_hot(target,num_classes = 72).float()

            output = output.reshape(-1, 72)

            expected = expected.reshape(-1,72)

            loss = criterion(output, expected)

            loss.backward()  # compute gradients
            nn.utils.clip_grad_norm_(model.parameters(),1)
            optimizer.step()  # update parameters

#                 total_loss += loss.item()

#                 break
#             continue
        with torch.no_grad():
            model.eval()
            for val_input, val_target in val_loader:
                val_input = val_input.to(device)
                val_target = val_target.to(device)
                val_output,_ = model.forward(val_input, None,False)

                acc_output = F.softmax(val_output,dim=2)
                acc_output = torch.argmax(acc_output,dim=2)
                acc_output = acc_output.T
                val_score += scoring(acc_output,val_target)


                val_output = val_output.permute(1, 0, 2)
                expected = F.one_hot(val_target,num_classes = 72).float()

                val_output = val_output.reshape(-1, 72)

                expected = expected.reshape(-1,72)


                loss = criterion(val_output, expected)
                val_loss += loss.item()

        with torch.no_grad():
            model.eval()
            for train_input, train_target in data_loader:
                train_input = train_input.to(device)
                train_target = train_target.to(device)
                train_output,_ = model.forward(train_input, None,False)

                acc_output = F.softmax(train_output,dim=2)
                acc_output = torch.argmax(acc_output,dim=2)
                acc_output = acc_output.T
                train_score += scoring(acc_output,train_target)


                train_output = train_output.permute(1, 0, 2)
                expected = F.one_hot(train_target,num_classes = 72).float()

                train_output = train_output.reshape(-1, 72)

                expected = expected.reshape(-1,72)


                loss = criterion(train_output, expected)
                train_loss += loss.item()
            model.train()



        print(f'epoch {epoch}')
        print(f'train loss => {train_loss/len(data_loader)} \ntrain_acc => {train_score/len(data_loader.dataset)}')
        print(f'valid loss => {val_loss/len(val_loader)} \nvalid_acc => {val_score/len(val_loader.dataset)}')
        train_loss_list.append(train_loss/len(data_loader))
        val_loss_list.append(val_loss/len(val_loader))
        train_accuracy_list.append(train_score/len(data_loader.dataset))
        val_accuracy_list.append(val_score/len(val_loader.dataset))

    return train_loss_list,val_loss_list,train_accuracy_list,val_accuracy_list    

# model = Seq2Seq()
# model.to(device)
# epochs = 2
# runModel(model, train_loader, val_loader ,epochs)

## WANDB SWEEPS TRAIN FUNCTION

In [53]:
def train_wandb():

    wandb.init(project="dl-assignment-3-attention", resume=True)

#     num_hidden_layer = wandb.config.num_hidden_layer

    wandb.run.name = f'inp_embed_{wandb.config.input_embedding}_enclayer_{wandb.config.number_of_enc_layer}_declayer_{wandb.config.number_of_dec_layer}_hidden_{wandb.config.hidden_size}_cell_{wandb.config.cell_type}_drop_{wandb.config.dropout}'

    model = Seq2Seq(   
        encoder_hidden_dimension = wandb.config.hidden_size, 
        decoder_hidden_dimension = wandb.config.hidden_size,
        encoder_embed_dimension =  wandb.config.input_embedding, 
        decoder_embed_dimension =  wandb.config.input_embedding, 
        bidirectional = wandb.config.bidirectional,
        encoder_num_layers = wandb.config.number_of_enc_layer,
        decoder_num_layers = wandb.config.number_of_dec_layer,
        rnn_type = wandb.config.cell_type, 
        dropout = wandb.config.dropout, 
        device = device,
        attention = True
    )
    model.to(device)
    epochs = 20
    train_loss_list,val_loss_list,train_accuracy_list,val_accuracy_list = runModel(model, train_loader, val_loader, epochs)
    #   print(val_loss_list,val_accuracy_list,train_loss_list,train_accuracy_list)
    for i in range(len(val_loss_list)):
        wandb.log({'validation_loss': val_loss_list[i],
                  'training_loss': train_loss_list[i],
                  'validation_accuracy': val_accuracy_list[i],
                  'training_accuracy': train_accuracy_list[i]
                  })

## WANDB SWEEP CONFIGURATIONS

In [ ]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'dl-assignment-3-attention',
    'metric': {
        'goal': 'maximize', 
        'name': 'validation_accuracy'
        },
    'parameters': {
        'input_embedding': {'values': [128, 256, 512]},
        'number_of_enc_layer': {'values': [2, 3]},
        'number_of_dec_layer': {'values': [2, 3]},
        'hidden_size': {'values': [128, 256]},
        'cell_type': {'values': ['lstm','gru','rnn']},
        'dropout': {'values': [0.2,0.3, 0.4]},
        'bidirectional' : {'values' : [True,False]}
     }
}

wandb.login(key = 'c425b887e2c725018a7f3a772582610fa54ef52c')
sweep_id = wandb.sweep(sweep=sweep_configuration, project='dl-assignment-3-attention')
wandb.agent(sweep_id, function=train_wandb, count=100)
wandb.finish()

## TEST ACCURACY CALCULATIONS

In [ ]:
def testAccuracy(model, test_loader):
    model.eval()
    test_score = 0
    test_loss = 0
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    
    for test_input, test_target in test_loader:
        test_input = test_input.to(device)
        test_target = test_target.to(device)
        test_output,_ = model.forward(test_input, None,False)

        acc_output = F.softmax(test_output,dim=2)
        acc_output = torch.argmax(acc_output,dim=2)
        acc_output = acc_output.T
        test_score += scoring(acc_output,test_target)


        test_output = test_output.permute(1, 0, 2)
        expected = F.one_hot(test_target,num_classes = 72).float()

        test_output = test_output.reshape(-1, 72)

        expected = expected.reshape(-1,72)


        loss = criterion(test_output, expected)
        test_loss += loss.item()
        
    print(f'test loss => {test_loss/len(test_loader)} \ntest_acc => {test_score/len(test_loader.dataset)}')


testAccuracy(model, test_loader)